In [ ]:
import cv2
import matplotlib
import numpy as np
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from skimage.metrics import mean_squared_error, structural_similarity as ssim

In [ ]:
# 1. Загрузка изображения
img = cv2.imread("sar_1_gray.jpg", cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap="gray")
plt.title("Исходное изображение")
plt.axis("off")
plt.show()

![Изображение](Figure_1.png)

In [ ]:
# 2. Построение гистограммы
plt.figure()
plt.title("Гистограмма исходного изображения")
plt.hist(img.ravel(), 256, [0, 256])
plt.xlabel("Яркость")
plt.ylabel("Количество пикселей")
plt.show()

![Изображение](Figure_2.png)

In [ ]:
# Кумулятивная гистограмма 
b_hist, bins = np.histogram(img.flatten(), 256, [0, 256])
b_hist_cum = b_hist.cumsum()
plt.figure()
plt.title("Кумулятивная гистограмма (CDF)")
plt.plot(b_hist_cum, color='red')
plt.xlabel("Яркость")
plt.ylabel("Накопленное количество пикселей")
plt.show()

![Изображение](Figure_2_1.png)

In [ ]:
# Нормированная гистограмма
b_hist_norm = b_hist / (img.shape[0] * img.shape[1])
plt.figure()
plt.title("Нормированная гистограмма (PDF)")
plt.plot(b_hist_norm, color='blue')
plt.xlabel("Яркость")
plt.ylabel("Вероятность")
plt.show()

![Изображение](Figure_2_2.png)

In [ ]:
# 3. Гамма-коррекция

# gamma < 1
gamma_low = 0.5
inv_gamma_low = 1.0 / gamma_low
table_low = np.array([(i / 255.0) ** inv_gamma_low * 255 for i in np.arange(256)]).astype("uint8")
gamma_img_low = cv2.LUT(img, table_low)

# gamma > 1
gamma_high = 1.5
inv_gamma_high = 1.0 / gamma_high
table_high = np.array([(i / 255.0) ** inv_gamma_high * 255 for i in np.arange(256)]).astype("uint8")
gamma_img_high = cv2.LUT(img, table_high)

# визуализация
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Исходное")
plt.imshow(img, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Gamma < 1")
plt.imshow(gamma_img_low, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Gamma > 1")
plt.imshow(gamma_img_high, cmap="gray")
plt.axis("off")
plt.show()

![Изображение](Figure_3.png)

In [ ]:
# 4. Сравнение изображений
mse_low = mean_squared_error(img, gamma_img_low)
ssim_low = ssim(img, gamma_img_low)

mse_high = mean_squared_error(img, gamma_img_high)
ssim_high = ssim(img, gamma_img_high)

print(f"[Gamma < 1] MSE: {mse_low:.4f}, SSIM: {ssim_low:.4f}")
print(f"[Gamma > 1] MSE: {mse_high:.4f}, SSIM: {ssim_high:.4f}")

[Gamma < 1] MSE: 2383.7636, SSIM: 0.5270 

[Gamma > 1] MSE: 1114.4519, SSIM: 0.8940

In [ ]:
# 5. Статистическая цветокоррекция на основе эталона eq_gray.jpg
image = cv2.imread("sar_1_gray.jpg")
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

# eq gray
eq_gray = cv2.equalizeHist(image_gray)

# Строим статистическую коррекцию
src_hist, _ = np.histogram(image_gray.flatten(), 256, [0, 256])
ref_hist, _ = np.histogram(eq_gray.flatten(), 256, [0, 256])

src_cdf = src_hist.cumsum() / src_hist.sum()
ref_cdf = ref_hist.cumsum() / ref_hist.sum()

mapping = np.interp(src_cdf, ref_cdf, np.arange(256))
matched = cv2.LUT(image_gray, mapping.astype('uint8'))

# Визуализация
plt.figure(figsize=(10, 4))
plt.subplot(1, 3, 1)
plt.title("Исходное")
plt.imshow(image_gray, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("eq gray")
plt.imshow(eq_gray, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("После цветокоррекции")
plt.imshow(matched, cmap="gray")
plt.axis("off")
plt.tight_layout()
plt.show()

![Изображение](Figure_5.png)

In [ ]:
# 6. Тест пороговой фильтрации
thresholds = [
    ("BINARY", cv2.THRESH_BINARY),
    ("BINARY_INV", cv2.THRESH_BINARY_INV),
    ("TRUNC", cv2.THRESH_TRUNC),
    ("TOZERO", cv2.THRESH_TOZERO),
    ("TOZERO_INV", cv2.THRESH_TOZERO_INV),
]

plt.figure(figsize=(10, 6))
for i, (name, t_type) in enumerate(thresholds, start=1):
    _, th = cv2.threshold(img, 127, 255, t_type)
    plt.subplot(2, 3, i)
    plt.imshow(th, cmap="gray")
    plt.title(name)
    plt.axis("off")

plt.tight_layout()
plt.show()

![Изображение](Figure_6.png)